In [ ]:
#install the packages using pip
!pip install --quiet git+https://github.com/huggingface/diffusers.git@d420d71398d9c5a8d9a5f95ba2bdb6fe3d8ae31f
!pip install --quiet ipython-autotime
!pip install --quiet transformers==4.34.1 accelerate==0.24.0 safetensors==0.4.0
!pip install --quiet ipyplot
!pip install gradio
%load_ext autotime

In [ ]:
#import the necessary libraries
import gradio as gr
from diffusers import StableDiffusionXLPipeline, StableDiffusionPipeline, DiffusionPipeline 
import torch
import ipyplot
import time

In [ ]:
#create pipeline for ssd-1b
pipe = StableDiffusionXLPipeline.from_pretrained("segmind/SSD-1B", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")


prompt = "Cinematic, breathtaking, sleek, a white elegant cat amidst a dense foggy London alley, poised beneath a lamp post, Striking image, 8K, Desktop background, Immensely sharp."
neg_prompt = "ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, blurry, bad anatomy, blurred, watermark, grainy, signature, cut off, draft"

image = pipe(prompt=prompt, negative_prompt=neg_prompt).images[0]

image.save("test.jpg")

ipyplot.plot_images([image],img_width=400)

In [ ]:
prompt = "The Renaissance Astrounaut"
neg_prompt = "ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, blurry, bad anatomy, blurred, watermark, grainy, signature, cut off, draft"

allimages = pipe(prompt=prompt, negative_prompt=neg_prompt,
                 guidance_scale=7.5,num_inference_steps=30,
                 num_images_per_prompt=2).images

images = [image for image in allimages]

for idx, image_file in enumerate(allimages, 1):
    image_file.save(f"test{idx}.jpg")

ipyplot.plot_images(images,img_width=400)

In [ ]:
#create a function to use in the gradio web ui
def gen_image(text, neg_prompt):
    return pipe(text,
                negative_prompt=neg_prompt,
                guidance_scale=7.5,
                num_inference_steps=30).images[0]

In [ ]:
#use the gen function 
gen_image("an orange cat staring off with pretty eyes", 
         "ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, 
          "out of frame, extra limbs, disfigured, deformed, body out of frame, blurry, 
          bad anatomy, blurred, watermark, grainy, signature, cut off, draft")

In [ ]:
#create a gradio web ui
txt = gr.Textbox(label="prompt")
txt_2 = gr.Textbox(label="neg_prompt")

demo = gr.Interface(fn=gen_image, inputs=[txt, txt_2], 
                    outputs="image", title = "Generate A.I.image using Distill Stable Diffusion😁")

demo.launch(share=True)

In [ ]:
#create the pipeline for all the 4 diffusion model
ssd_1b = StableDiffusionXLPipeline.from_pretrained(
    "segmind/SSD-1B", torch_dtype=torch.float16, use_safetensors=True, 
    variant="fp16").to("cuda")

distilled = StableDiffusionPipeline.from_pretrained(
    "nota-ai/bk-sdm-small", torch_dtype=torch.float16, use_safetensors=True,
).to("cuda")

original = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16, use_safetensors=True,
).to("cuda")

SDXL_Original = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, 
    use_safetensors=True, variant="fp16").to("cuda")

In [ ]:
seed = 2023
generator = torch.manual_seed(seed)

NUM_ITERS_TO_RUN = 3
NUM_INFERENCE_STEPS = 25
NUM_IMAGES_PER_PROMPT = 4

prompt = "The Robotic Baroque Battle"

start = time.time_ns()
for _ in range(NUM_ITERS_TO_RUN):
    images = ssd_1b(
        prompt,
        num_inference_steps=NUM_INFERENCE_STEPS,
        generator=generator,
        num_images_per_prompt=NUM_IMAGES_PER_PROMPT
    ).images
end = time.time_ns()
original_sd = f"{(end - start) / 1e6:.1f}"

print(f"Execution time -- {original_sd} ms\n")

In [ ]:
for image in images:
    ipyplot.plot_images([image],img_width=400)

In [ ]:
seed = 2023
generator = torch.manual_seed(seed)

NUM_ITERS_TO_RUN = 3
NUM_INFERENCE_STEPS = 25
NUM_IMAGES_PER_PROMPT = 4

prompt = "The Robotic Baroque Battle"

start = time.time_ns()
for _ in range(NUM_ITERS_TO_RUN):
    images = SDXL_Original(
        prompt,
        num_inference_steps=NUM_INFERENCE_STEPS,
        generator=generator,
        num_images_per_prompt=NUM_IMAGES_PER_PROMPT
    ).images
end = time.time_ns()
original_sd = f"{(end - start) / 1e6:.1f}"

print(f"Execution time -- {original_sd} ms\n")

In [ ]:
for image in images:
    ipyplot.plot_images([image],img_width=400)

In [ ]:
seed = 2023
generator = torch.manual_seed(seed)

NUM_ITERS_TO_RUN = 3
NUM_INFERENCE_STEPS = 25
NUM_IMAGES_PER_PROMPT = 4

prompt = "The Robotic Baroque Battle"

start = time.time_ns()
for _ in range(NUM_ITERS_TO_RUN):
    images = original(
        prompt,
        num_inference_steps=NUM_INFERENCE_STEPS,
        generator=generator,
        num_images_per_prompt=NUM_IMAGES_PER_PROMPT
    ).images
end = time.time_ns()
original_sd = f"{(end - start) / 1e6:.1f}"

print(f"Execution time -- {original_sd} ms\n")

In [ ]:
for image in images:
    ipyplot.plot_images([image],img_width=400)

In [ ]:
seed = 2023
generator = torch.manual_seed(seed)

NUM_ITERS_TO_RUN = 3
NUM_INFERENCE_STEPS = 25
NUM_IMAGES_PER_PROMPT = 4

prompt = "The Robotic Baroque Battle"

start = time.time_ns()
for _ in range(NUM_ITERS_TO_RUN):
    images = distilled(
        prompt,
        num_inference_steps=NUM_INFERENCE_STEPS,
        generator=generator,
        num_images_per_prompt=NUM_IMAGES_PER_PROMPT
    ).images
end = time.time_ns()

distilled_sd = f"{(end - start) / 1e6:.1f}"
print(f"Execution time -- {distilled_sd} ms\n")

In [ ]:
for image in images:
    ipyplot.plot_images([image],img_width=400)